<a href="https://colab.research.google.com/github/talal777-ye/for-Collage/blob/main/ZKP_Lab__TalalBadubbah_22202041052.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python3
"""
schnorr_zkp_demo.py
Demonstration of:
  - Interactive Schnorr ZKP (honest prover)
  - Cheating prover simulation (estimation of success probability)
  - Fiat-Shamir transform (non-interactive) and forging attempts
Logs all steps and prints summaries.

Educational/demo code - not for production/real-security usage.
"""

import random
import hashlib

# -----------------------------
# Group parameters (small demo)
# -----------------------------
# We'll use a small safe-like prime p where q = (p-1)//2 is prime (for demo).
# Example: p = 23, q = 11, generator g = 2 or 5 are primitive roots for p=23.
p = 23
q = 11
g = 2  # generator of subgroup of order q (works for this small example)

# Check basic properties (debug)
assert (p - 1) % q == 0, "Ensure q divides p-1"
# Note: in a real setup choose p, q properly (safe prime) and g of order q.

# -----------------------------
# Key generation (Prover's secret)
# -----------------------------
# Secret x in Z_q, public y = g^x mod p
def keygen():
    x = random.randint(1, q-1)  # secret
    y = pow(g, x, p)
    return x, y

# -----------------------------
# Schnorr interactive protocol
# -----------------------------
# Protocol:
# Prover chooses random r, sends t = g^r mod p
# Verifier sends random challenge e in Z_q
# Prover responds s = r + e*x (mod q)
# Verifier checks: g^s == t * y^e (mod p)

def prover_commit(x=None, r=None):
    """Prover commitment: choose r and return t and r."""
    if r is None:
        r = random.randint(0, q-1)
    t = pow(g, r, p)
    return t, r

def prover_respond(r, e, x):
    """Compute s = r + e*x (mod q)"""
    return (r + e * x) % q

def verifier_check(t, s, y, e):
    """Check g^s == t * y^e (mod p)"""
    left = pow(g, s, p)
    right = (t * pow(y, e, p)) % p
    return left == right

# -----------------------------
# Cheating prover (interactive)
# -----------------------------
# Strategy: cheating prover who doesn't know x can precommit to a pair (t,e0,s0)
# by choosing an e0 in advance and computing s0 randomly and setting t = g^s0 * y^{-e0}.
# If the verifier's random e equals e0, cheat succeeds. So probability ≈ 1/q.
#
# Implementation:
def cheating_precompute_pair(e0=None):
    """Cheating prover precomputes (t, s0, e0) so that if verifier challenges with e0, success."""
    if e0 is None:
        e0 = random.randint(0, q-1)
    s0 = random.randint(0, q-1)
    # compute t = g^s0 * y^{-e0} mod p
    # But cheating prover does not know y^(-e0) if they don't know y? In practice
    # the public key y is public, so they can compute t using y.
    # t = g^s0 * inv(y^e0) mod p  => same as g^s0 * y^{-e0}
    # Use pow for inverse: y^{-e0} = pow(y, q - e0, p) if q is order, but general inverse mod p:
    # compute t straightforwardly:
    return s0, e0  # t will be computed later with known y when needed

def cheating_compute_t_from_s_e(s0, e0, y):
    # t = g^s0 * y^{-e0} (mod p)
    # compute y^{-e0} as pow(y, -e0, p) using pow with exponent p-1-e0 mod (p-1) won't work directly;
    # easier: compute inv = modular inverse of pow(y, e0, p) mod p
    ye = pow(y, e0, p)
    inv_ye = pow(ye, -1, p)  # Python 3.8+ supports negative exponent -> modular inverse
    t = (pow(g, s0, p) * inv_ye) % p
    return t

# -----------------------------
# Fiat-Shamir transform (non-interactive)
# -----------------------------
# e = H(t || message) mod q
def fiat_shamir_challenge(t, message=b""):
    # create canonical bytes for t
    t_bytes = str(t).encode()
    h = hashlib.sha256(t_bytes + message).digest()
    e = int.from_bytes(h, 'big') % q
    return e

# -----------------------------
# Experiments and logging
# -----------------------------
def run_honest_interactive_demo():
    print("=== Honest Interactive Schnorr Demo ===")
    x, y = keygen()
    print(f"Secret x = {x}, Public y = {y}")
    t, r = prover_commit(x)
    print(f"Prover sends commitment t = {t}")
    e = random.randint(0, q-1)
    print(f"Verifier random challenge e = {e}")
    s = prover_respond(r, e, x)
    print(f"Prover responds s = {s}")
    ok = verifier_check(t, s, y, e)
    print("Verifier check:", "ACCEPT" if ok else "REJECT")
    return ok

def run_cheating_interactive_experiment(trials=1000):
    print("\n=== Cheating Prover Experiment (Interactive) ===")
    x, y = keygen()  # honest keypair (cheater knows not x)
    print(f"(Public key y = {y})")
    success = 0
    for i in range(trials):
        # Cheater precomputes an (s0, e0) pair
        s0 = random.randint(0, q-1)
        e0 = random.randint(0, q-1)
        t = cheating_compute_t_from_s_e(s0, e0, y)
        # Verifier chooses random e
        e = random.randint(0, q-1)
        # Cheater will respond with s0 (only works if e == e0)
        if e == e0:
            # verifier check would accept
            if verifier_check(t, s0, y, e):
                success += 1
        else:
            # in general verifier_check will fail
            pass
    prob = success / trials
    print(f"Trials = {trials}, successes = {success}, estimated cheating probability = {prob:.6f}")
    print(f"Theoretical ~ 1/q = {1/q:.6f}")
    return prob

def run_fiat_shamir_demo_and_forgery(trials=1000, forge_attempts=50, message=b"txn:Send 100 coins to Ali"):
    print("\n=== Fiat–Shamir Non-Interactive Demo & Forgery Attempts ===")
    x, y = keygen()
    print(f"Secret x = {x}, Public y = {y}")
    # Honest (non-interactive) proof generation:
    t, r = prover_commit(x)
    e = fiat_shamir_challenge(t, message)
    s = prover_respond(r, e, x)
    ok = verifier_check(t, s, y, e)
    print("\nHonest Fiat-Shamir proof produced.")
    print(f"t={t}, e={e}, s={s}, verifier: {'ACCEPT' if ok else 'REJECT'}")

    # Forgery (cheater tries to produce (t,s) such that e = H(t||m) and verifier accepts)
    # Cheater strategy: choose s0 and e0, compute t = g^s0 * y^{-e0}, then hope H(t||m) == e0.
    # The chance that H(t||m) equals the pre-chosen e0 is ≈ 1/q for each random t.
    print(f"\nAttempting forgery: trials = {trials}, attempts per trial = {forge_attempts}")
    total_success = 0
    for i in range(trials):
        forged = False
        for attempt in range(forge_attempts):
            s0 = random.randint(0, q-1)
            e0 = random.randint(0, q-1)
            t = cheating_compute_t_from_s_e(s0, e0, y)
            e_hash = fiat_shamir_challenge(t, message)
            if e_hash == e0:
                # verifier_check would accept:
                if verifier_check(t, s0, y, e0):
                    forged = True
                    break
        if forged:
            total_success += 1
    prob_per_trial = total_success / trials
    print(f"Forged successes = {total_success} / {trials} => success prob ≈ {prob_per_trial:.6f}")
    # Theoretical per single attempt success ~ 1/q, so per trial with A attempts ≈ 1 - (1-1/q)^A
    approx = 1 - (1 - 1/q) ** forge_attempts
    print(f"Theoretical approx success per trial with {forge_attempts} attempts ≈ {approx:.6f}")
    return prob_per_trial

# -----------------------------
# Run all demos & experiments
# -----------------------------
if __name__ == "__main__":
    random.seed(42)  # for reproducible demo runs; remove for varied results

    # 1) Honest interactive demo
    _ = run_honest_interactive_demo()

    # 2) Cheating prover experiment (interactive)
    TRIALS = 2000
    prob_cheat = run_cheating_interactive_experiment(trials=TRIALS)

    # 3) Fiat–Shamir non-interactive and forging experiment
    FS_TRIALS = 1000
    ATTEMPTS_PER_TRIAL = 30
    prob_forge = run_fiat_shamir_demo_and_forgery(trials=FS_TRIALS, forge_attempts=ATTEMPTS_PER_TRIAL)

    # Summary
    print("\n=== Summary ===")
    print(f"Group params: p={p}, q={q}, g={g}")
    print(f"Interactive cheating estimated prob ≈ {prob_cheat:.6f} (theoretical 1/q = {1/q:.6f})")
    print(f"Fiat-Shamir forging prob per trial (with {ATTEMPTS_PER_TRIAL} attempts) ≈ {prob_forge:.6f}")


=== Honest Interactive Schnorr Demo ===
Secret x = 2, Public y = 4
Prover sends commitment t = 1
Verifier random challenge e = 4
Prover responds s = 8
Verifier check: ACCEPT

=== Cheating Prover Experiment (Interactive) ===
(Public key y = 16)
Trials = 2000, successes = 175, estimated cheating probability = 0.087500
Theoretical ~ 1/q = 0.090909

=== Fiat–Shamir Non-Interactive Demo & Forgery Attempts ===
Secret x = 7, Public y = 13

Honest Fiat-Shamir proof produced.
t=16, e=6, s=2, verifier: ACCEPT

Attempting forgery: trials = 1000, attempts per trial = 30
Forged successes = 946 / 1000 => success prob ≈ 0.946000
Theoretical approx success per trial with 30 attempts ≈ 0.942691

=== Summary ===
Group params: p=23, q=11, g=2
Interactive cheating estimated prob ≈ 0.087500 (theoretical 1/q = 0.090909)
Fiat-Shamir forging prob per trial (with 30 attempts) ≈ 0.946000
